# YOLO Training Interface in Google Colab
This notebook demonstrates how to set up and run the YOLO Training Interface in Google Colab.

## Step 1: Install Dependencies

In [ ]:
# Install the required dependencies
!python -m pip install ultralytics>=8.0.0 rich>=10.0.0 pyyaml>=6.0 pandas>=1.3.0 matplotlib>=3.4.0 numpy>=1.20.0 ray>=2.0.0 gdown>=4.6.0 opencv-python>=4.5.0

# Import necessary packages for this notebook
import os
import yaml

## Step 2: Upload the required files
Upload the following files to this Colab session:
1. program.py - The main YOLO training interface
2. setup_env.py - The environment setup script

In [ ]:
# Download files directly from Github repository
!wget https://raw.githubusercontent.com/d-quint/colab/refs/heads/main/program.py -O program.py
!wget https://raw.githubusercontent.com/d-quint/colab/refs/heads/main/setup_env.py -O setup_env.py

# Verify the files were downloaded correctly
!ls -la program.py setup_env.py

## Step 3: Set up the Environment

In [ ]:
# Run the setup script to create the necessary directory structure
%run setup_env.py --download_models

## Fix Sample Model File

The setup script creates a placeholder .pt file that isn't a real model. Let's replace it with a real YOLOv8 model.

In [ ]:
# Fix the sample weight file by replacing it with a real YOLOv8 model
import os
from ultralytics import YOLO

# Directory for the sample model
sample_weights_dir = "/content/drive/MyDrive/YOLO_Training/runs/detect/yolov8n_sample/weights"
sample_weights_file = os.path.join(sample_weights_dir, "last.pt")

# Check if the sample weight file exists and is just a placeholder
if os.path.exists(sample_weights_file):
    with open(sample_weights_file, 'r') as f:
        content = f.read()
        if content.startswith("#"):
            print("Detected placeholder model file. Replacing with a real YOLOv8n model...")

            # Download a real YOLOv8n model
            model = YOLO("yolov8n.pt")

            # Save the real model to the sample weights directory
            os.remove(sample_weights_file)  # Remove the placeholder
            model.save(sample_weights_file)  # Save the real model

            print(f"✅ Successfully replaced placeholder with real model at: {sample_weights_file}")
else:
    print(f"Sample weights directory not found. Creating it...")
    os.makedirs(sample_weights_dir, exist_ok=True)

    # Download and save a real YOLOv8n model
    model = YOLO("yolov8n.pt")
    model.save(sample_weights_file)

    print(f"✅ Created real model at: {sample_weights_file}")

## Step 4: Download Pretrained Model
Download a pretrained YOLO model from Google Drive. This model (new3ktrash_tip) contains:
- `args.yaml` - The arguments/configuration used for training the model
- `weights/best.pt` - The trained model weights

We'll add this model to the YOLO environment so it can be used for inference or further training.
Note: This is a custom trained model, not just a dataset. After downloading, we'll be able to use it for inference on new images or fine-tune it further.

In [16]:
# Download pretrained model from Google Drive using gdown
import gdown
import os
import shutil
import yaml
from pathlib import Path

# Google Drive file ID for the new3ktrash_tip model
file_id = "19Mzi4Ueqyh3i634c9oUvP4NEskwoB86z"
output_path = "pretrained_model.zip"
model_name = "new3ktrash_tip"  # Name of the model when extracted

# Target directory in the runs/detect folder where models are stored
model_dir = f"/content/drive/MyDrive/YOLO_Training/runs/detect/{model_name}"
model_weights_dir = os.path.join(model_dir, "weights")

# Check if the model is already downloaded
if os.path.exists(os.path.join(model_weights_dir, "best.pt")):
    print(f"✅ Model {model_name} is already downloaded and ready to use.")
else:
    # Download the file
    print(f"Downloading pretrained model '{model_name}' from Google Drive...")
    gdown.download(id=file_id, output=output_path, quiet=False)

    # Extract the model
    print("\nExtracting the model files...")

    # Create the target directories
    os.makedirs(model_weights_dir, exist_ok=True)

    # Extract the model archive
    !unzip -q {output_path} -d /tmp/model_extract

    # Check what was extracted and move it to the correct location
    extract_dir = "/tmp/model_extract"
    extract_contents = os.listdir(extract_dir)
    print(f"Extracted contents: {extract_contents}")

    # Handle different archive structures
    if model_name in extract_contents:
        # If the archive contains a directory with the model name
        src_dir = os.path.join(extract_dir, model_name)
        if os.path.isdir(src_dir):
            print(f"Moving files from {src_dir} to {model_dir}")
            !cp -r {src_dir}/* {model_dir}
    else:
        # If the archive extracts directly to files
        print(f"Moving all extracted files to {model_dir}")
        !cp -r {extract_dir}/* {model_dir}

    # Clean up the temporary files
    !rm {output_path}
    !rm -rf /tmp/model_extract

# Verify the model structure
print(f"\nVerifying downloaded model structure:")
!ls -la {model_dir}

# Check for weights directory
if os.path.exists(model_weights_dir):
    print("\nModel weights directory:")
    !ls -la {model_weights_dir}
else:
    print("\n❌ Warning: Weights directory not found!")
    # Try to find weight files in the main directory
    weight_files = [f for f in os.listdir(model_dir) if f.endswith('.pt')]
    if weight_files:
        print(f"Found weight files in main directory: {weight_files}")
        # Create weights directory and move files
        os.makedirs(model_weights_dir, exist_ok=True)
        for wf in weight_files:
            shutil.move(os.path.join(model_dir, wf), os.path.join(model_weights_dir, wf))
        print("✅ Moved weight files to weights directory")
        !ls -la {model_weights_dir}

# Verify the key files exist
args_yaml = os.path.join(model_dir, "args.yaml")
best_weights = os.path.join(model_weights_dir, "best.pt")
last_weights = os.path.join(model_weights_dir, "last.pt")

# Check for configuration file
if os.path.exists(args_yaml):
    print(f"\n✅ Found model configuration: {args_yaml}")
    # Display the configuration
    with open(args_yaml, 'r') as f:
        try:
            config = yaml.safe_load(f)
            print("\nModel Configuration:")
            for key, value in config.items():
                print(f"  {key}: {value}")
        except Exception as e:
            print(f"❌ Error reading config: {str(e)}")
else:
    print(f"\n❌ Warning: Model configuration not found at: {args_yaml}")

# Check for weight files
weight_found = False
if os.path.exists(best_weights):
    print(f"✅ Found best model weights: {best_weights}")
    primary_weights = best_weights
    weight_found = True
elif os.path.exists(last_weights):
    print(f"✅ Found last model weights: {last_weights}")
    primary_weights = last_weights
    weight_found = True
else:
    # Check for any PT files
    weight_files = [f for f in os.listdir(model_weights_dir) if f.endswith('.pt')]
    if weight_files:
        primary_weights = os.path.join(model_weights_dir, weight_files[0])
        print(f"✅ Found alternative weight file: {weight_files[0]}")
        weight_found = True
    else:
        print(f"❌ Error: No model weight files found!")
        primary_weights = None

# Save paths for later use
if weight_found:
    # Save the model path for easy access in other cells
    model_path_file = "/content/drive/MyDrive/YOLO_Training/model_path.txt"
    with open(model_path_file, 'w') as f:
        f.write(primary_weights)
    print(f"\n✅ Model is ready to use at: {primary_weights}")
    print(f"   Model path saved to: {model_path_file}")

✅ Model new3ktrash_tip is already downloaded and ready to use.

Verifying downloaded model structure:
total 9954
-rw------- 1 root root    1651 May 19 02:35 args.yaml
-rw------- 1 root root   85212 May 19 02:35 confusion_matrix_normalized.png
-rw------- 1 root root   89868 May 19 02:35 confusion_matrix.png
-rw------- 1 root root   93509 May 19 02:35 F1_curve.png
-rw------- 1 root root  228215 May 19 02:35 labels_correlogram.jpg
-rw------- 1 root root  213570 May 19 02:35 labels.jpg
-rw------- 1 root root   88320 May 19 02:35 P_curve.png
-rw------- 1 root root   83255 May 19 02:35 PR_curve.png
-rw------- 1 root root   93882 May 19 02:35 R_curve.png
-rw------- 1 root root    6201 May 19 02:35 results.csv
-rw------- 1 root root  287520 May 19 02:35 results.png
-rw------- 1 root root  706234 May 19 02:35 train_batch0.jpg
-rw------- 1 root root  674445 May 19 02:35 train_batch1.jpg
-rw------- 1 root root  664638 May 19 02:35 train_batch2.jpg
-rw------- 1 root root  587790 May 19 02:35 train

In [17]:
# Update the dataset.yaml file to point to the downloaded dataset
import yaml
import os

# Path where we want to save our dataset.yaml
dataset_yaml = "/content/drive/MyDrive/YOLO_Training/dataset.yaml"

# Dataset directory setup
custom_dataset_dir = "/content/drive/MyDrive/YOLO_Training/datasets/custom_dataset"

# Check if the dataset directory exists, create it if it doesn't
if not os.path.exists(custom_dataset_dir):
    print(f"Dataset directory not found. Creating: {custom_dataset_dir}")
    os.makedirs(custom_dataset_dir, exist_ok=True)
    os.makedirs(os.path.join(custom_dataset_dir, "images", "train"), exist_ok=True)
    os.makedirs(os.path.join(custom_dataset_dir, "images", "val"), exist_ok=True)
    os.makedirs(os.path.join(custom_dataset_dir, "images", "test"), exist_ok=True)
    os.makedirs(os.path.join(custom_dataset_dir, "labels", "train"), exist_ok=True)
    os.makedirs(os.path.join(custom_dataset_dir, "labels", "val"), exist_ok=True)
    os.makedirs(os.path.join(custom_dataset_dir, "labels", "test"), exist_ok=True)
    print(f"✅ Created sample dataset directory structure")

# Look for yaml files in the dataset directory
yaml_files = []
if os.path.exists(custom_dataset_dir):
    yaml_files = [f for f in os.listdir(custom_dataset_dir) if f.endswith('.yaml')]

if yaml_files:
    # Use the first yaml file found
    dataset_config_path = os.path.join(custom_dataset_dir, yaml_files[0])
    print(f"Found dataset config: {dataset_config_path}")

    # Read the yaml file
    with open(dataset_config_path, 'r') as f:
        data = yaml.safe_load(f)

    # Update paths to be absolute
    data['path'] = custom_dataset_dir

    # Save to our main dataset.yaml location
    with open(dataset_yaml, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)

    print(f"Updated {dataset_yaml} with dataset configuration")
    print("\nDataset configuration:")
    with open(dataset_yaml, 'r') as f:
        print(f.read())
else:
    print(f"No YAML configuration found. Creating a default dataset.yaml file.")

    # Create a default dataset.yaml
    default_config = {
        'path': custom_dataset_dir,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {
            0: 'object',  # Default class name
        }
    }

    with open(dataset_yaml, 'w') as f:
        yaml.dump(default_config, f, default_flow_style=False)

    print("\nDefault dataset configuration created:")
    with open(dataset_yaml, 'r') as f:
        print(f.read())

    print("\nNote: You should update this default configuration with your actual classes.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/YOLO_Training/datasets/custom_dataset'

## Step 5: Run the YOLO Training Interface

In [ ]:
# Change to the directory where the program.py is located
%cd /content/drive/MyDrive/YOLO_Training

# Run the program
%run program.py

## Optional: Monitor Training with TensorBoard
YOLOv8 automatically creates TensorBoard logs which you can view in Colab.

In [ ]:
# Load the TensorBoard extension
%load_ext tensorboard

# Start TensorBoard
%tensorboard --logdir /content/drive/MyDrive/YOLO_Training/runs

## Troubleshooting

### Error: "invalid load key, '#'"

If you encounter the error message "invalid load key, '#'" when trying to run hyperparameter tuning, this typically happens because the setup script creates placeholder model files instead of real ones. The fix is included in this notebook (see "Fix Sample Model File" section above).

If you still encounter this issue:

1. Check if the model file is a real PyTorch model and not a placeholder:

In [ ]:
# Check if the model file is valid
def check_model_file(model_path):
    try:
        # Try opening as text
        with open(model_path, 'r') as f:
            content = f.read(100)  # Read just the first 100 chars

            # If it starts with a # character, it's a placeholder text file
            if content.strip().startswith('#'):
                print(f"❌ Error: {model_path} is a placeholder text file, not a real model.")
                print("Content preview:")
                print(content)
                return False
    except UnicodeDecodeError:
        # If we can't read it as text, it's likely a binary file (good!)
        print(f"✅ {model_path} appears to be a binary file (likely a real model).")
        return True

    # Final check - try loading with YOLO
    try:
        from ultralytics import YOLO
        model = YOLO(model_path)
        print(f"✅ Successfully loaded {model_path} as a YOLO model.")
        return True
    except Exception as e:
        print(f"❌ Error loading model with YOLO: {str(e)}")
        return False

# Check the sample model file
sample_model_path = "/content/drive/MyDrive/YOLO_Training/runs/detect/yolov8n_sample/weights/last.pt"
if os.path.exists(sample_model_path):
    check_model_file(sample_model_path)
else:
    print(f"Model file not found: {sample_model_path}")

2. If the model file is invalid, replace it with a real one:

In [ ]:
# Replace an invalid model file with a real one
def replace_invalid_model(model_path):
    print(f"Replacing invalid model at {model_path} with a real YOLOv8n model...")

    # Make sure directory exists
    os.makedirs(os.path.dirname(model_path), exist_ok=True)

    # Download a real YOLOv8n model
    from ultralytics import YOLO
    model = YOLO("yolov8n.pt")

    # Remove old file if it exists
    if os.path.exists(model_path):
        os.remove(model_path)

    # Save the real model
    model.save(model_path)
    print(f"✅ Successfully replaced with real model at: {model_path}")
    return True

# Uncomment and run if needed
# replace_invalid_model("/content/drive/MyDrive/YOLO_Training/runs/detect/yolov8n_sample/weights/last.pt")

## Step 4.1: Test the Downloaded Model
Let's verify the downloaded model works by performing inference on a sample image.

In [ ]:
# Test the downloaded model with a sample image
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Check if we have the model path from the previous step
try:
    with open("/content/drive/MyDrive/YOLO_Training/model_path.txt", 'r') as f:
        model_path = f.read().strip()
        print(f"Using model from: {model_path}")
except FileNotFoundError:
    # Use default path if file not found
    model_path = f"/content/drive/MyDrive/YOLO_Training/runs/detect/new3ktrash_tip/weights/best.pt"
    print(f"Using default model path: {model_path}")

# Check if model exists
if not os.path.exists(model_path):
    print(f"❌ Error: Model not found at {model_path}")
    # Try to find alternatives
    model_dir = os.path.dirname(os.path.dirname(model_path))
    weights_dir = os.path.join(model_dir, "weights")

    if os.path.exists(weights_dir):
        weight_files = [f for f in os.listdir(weights_dir) if f.endswith('.pt')]
        if weight_files:
            model_path = os.path.join(weights_dir, weight_files[0])
            print(f"Found alternative model: {model_path}")
        else:
            print("No model weight files found!")
    else:
        print(f"Weights directory not found at {weights_dir}")

# Load the model
try:
    model = YOLO(model_path)
    print("✅ Model loaded successfully!")

    # Download a sample image for testing
    !wget -q https://ultralytics.com/images/bus.jpg -O sample_image.jpg

    # Display the image
    img = cv2.imread('sample_image.jpg')
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    plt.title("Sample Image")
    plt.axis('off')
    plt.show()

    # Run inference
    print("\nRunning inference on sample image...")
    results = model('sample_image.jpg')

    # Display results
    print("\nDetection Results:")
    for result in results:
        boxes = result.boxes
        print(f"Found {len(boxes)} objects")

        # Get class names from the model
        class_names = result.names
        print("Classes detected:", [class_names[int(cls)] for cls in boxes.cls.cpu().numpy()])

    # Plot detection results
    plt.figure(figsize=(12, 9))
    result_img = results[0].plot()
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.title("Detection Results")
    plt.axis('off')
    plt.show()

    print("\n✅ Model test complete!")

except Exception as e:
    print(f"❌ Error loading or using the model: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Make sure the model file exists and is a valid YOLO model")
    print("2. Check if the model format is compatible with your version of ultralytics")
    print("3. Try downloading the model again")

## Step 4.2: Using the Pretrained Model in the Training Interface

When running the YOLO Training Interface in Step 5, you can use the downloaded model for:

1. **Inference**: Selecting the model in the interface to run predictions on new images
2. **Fine-tuning**: Further training the model on your own dataset
3. **Hyperparameter tuning**: Optimizing the model parameters for better performance

To use the downloaded model in the training interface:

1. When prompted to select a base model, choose **"Use previously trained custom model"**
2. Select the **"new3ktrash_tip"** directory from the list
3. Choose the weight file (usually **"best.pt"**) from the weights directory

You can also access the model directly in your code:

In [ ]:
# Example of how to load and use the downloaded model in your own code
from ultralytics import YOLO

# Load the model (this will use the path we saved earlier)
try:
    with open("/content/drive/MyDrive/YOLO_Training/model_path.txt", 'r') as f:
        model_path = f.read().strip()
except FileNotFoundError:
    model_path = "/content/drive/MyDrive/YOLO_Training/runs/detect/new3ktrash_tip/weights/best.pt"

# Create a YOLO model instance
model = YOLO(model_path)

print(f"Model loaded from: {model_path}")
print("\nExample usage:")
print("1. Run inference on an image:")
print('   results = model("path/to/image.jpg")')
print("2. Fine-tune the model on a new dataset:")
print('   results = model.train(data="dataset.yaml", epochs=50)')
print("3. Export the model for deployment:")
print('   success = model.export(format="onnx")')

### Error: "Access denied with the following error: ..."

If you encounter access denied errors when downloading from Google Drive:

1. The file sharing settings might have changed
2. Google Drive may be rate-limiting your downloads

Solutions:
- Try using the direct Google Drive web interface to download the file
- Create a shortcut to the file in your own Google Drive, then download from there

### Error: "Model file not found" or "No model weight files found"

If the model weights cannot be found after downloading:

1. Check the extracted directory structure
2. The archive might have a different structure than expected

In [ ]:
# Function to manually locate and fix model files
def find_and_fix_model_files():
    import os
    from pathlib import Path

    model_name = "new3ktrash_tip"
    base_dir = "/content/drive/MyDrive/YOLO_Training"
    model_dir = os.path.join(base_dir, "runs", "detect", model_name)

    print(f"Searching for model files in {model_dir}...")

    # Check if directory exists
    if not os.path.exists(model_dir):
        print(f"❌ Model directory not found: {model_dir}")
        return

    # Look for weights
    weight_files = []
    for root, dirs, files in os.walk(model_dir):
        for file in files:
            if file.endswith('.pt'):
                weight_files.append(os.path.join(root, file))

    if weight_files:
        print(f"✅ Found {len(weight_files)} weight files:")
        for i, wf in enumerate(weight_files):
            print(f"  [{i+1}] {wf}")

        # Create weights dir if needed
        weights_dir = os.path.join(model_dir, "weights")
        os.makedirs(weights_dir, exist_ok=True)

        # Move files that aren't already in the weights directory
        for wf in weight_files:
            if os.path.dirname(wf) != weights_dir:
                new_path = os.path.join(weights_dir, os.path.basename(wf))
                print(f"Moving {wf} → {new_path}")

                if not os.path.exists(new_path):
                    import shutil
                    shutil.copy2(wf, new_path)
                    print(f"  ✅ Copied to weights directory")

        # Verify weights directory
        print(f"\nWeights directory contents:")
        if os.path.exists(weights_dir):
            print(os.listdir(weights_dir))
        else:
            print("❌ Weights directory still not found")
    else:
        print("❌ No weight files found anywhere in the model directory!")

    # Check for args.yaml
    args_yaml = os.path.join(model_dir, "args.yaml")
    if os.path.exists(args_yaml):
        print(f"✅ Found args.yaml file: {args_yaml}")
    else:
        yaml_files = []
        for root, dirs, files in os.walk(model_dir):
            for file in files:
                if file.endswith('.yaml'):
                    yaml_files.append(os.path.join(root, file))

        if yaml_files:
            print(f"✅ Found {len(yaml_files)} YAML files:")
            for i, yf in enumerate(yaml_files):
                print(f"  [{i+1}] {yf}")

            # Copy the first yaml file to args.yaml if it doesn't exist
            import shutil
            shutil.copy2(yaml_files[0], args_yaml)
            print(f"  ✅ Copied {yaml_files[0]} to {args_yaml}")
        else:
            print("❌ No YAML configuration files found!")

            # Create a basic args.yaml
            import yaml
            basic_args = {
                'task': 'detect',
                'model': model_name,
                'data': 'dataset.yaml',
                'epochs': 100,
                'patience': 50,
                'batch': 16,
                'imgsz': 640,
                'save': True,
                'exist_ok': False,
                'name': model_name
            }

            with open(args_yaml, 'w') as f:
                yaml.dump(basic_args, f, default_flow_style=False)
            print(f"  ✅ Created a basic args.yaml file")

# Uncomment to run this function if you're having trouble with model files
# find_and_fix_model_files()